# Deploy Serverless endpoint - Object Detection (YOLO-v3)
---

***Note: 본 핸즈온에 사용된 추론 코드와 Dockerfile은 https://github.com/kts102121/lambda_container 에서 확인할 수 있으며, Lambda 추론에 대한 더 많은 예제들을 확인할 수 있습니다.***

## Overview

re:Invent 2020에 소개된 Lambda 컨테이너 기능 지원으로 기존 Lambda에서 수행하기 어려웠던 대용량 머신 모델에 대한 추론을 보다 수월하게 실행할 수 있게 되었습니다. Lambda 컨테이너 이미지를 Amazon ECR(Amazon Elastic Container Registry)에 푸시하였다면 Lambda 함수를 생성하여 직접 컨테이너 이미지를 배포하거나 SageMaker의 API 호출로 Serverless endpoint를 쉽게 배포할 수 있습니다.

자세한 내용은 아래 링크를 참조해 주세요.
- AWS Lambda의 새로운 기능 — 컨테이너 이미지 지원: https://aws.amazon.com/ko/blogs/korea/new-for-aws-lambda-container-image-support/
- SageMaker Serverless Inference: https://sagemaker.readthedocs.io/en/stable/overview.html?highlight=lambdamodel#serverless-inference
- AWS Builders Online - AWS Lambda 컨테이너 이미지 서비스 활용하기 (김태수 SA): https://www.youtube.com/watch?v=tTg9Lp7Sqok

<br>

## 1. Preparation
---

필요한 함수들을 정의하고 Serverless 추론에 필요한 권한을 아래와 같이 설정합니다. 참고로, 직접 Lambda Container 함수를 배포 시에는 ECR 리포지토리에 대한 억세스를 자동으로 생성해 줍니다.

- SageMaker과 연결된 role 대해 ECR 억세스를 허용하는 policy 생성 및 연결
- SageMaker 노트북에서 lambda를 실행할 수 있는 role 생성
- Lambda 함수가 ECR private 리포지토리에 연결하는 억세스를 허용하는 policy 생성 및 연결 

In [ ]:
import json
import time
import boto3
import sagemaker
import base64
from sagemaker import get_execution_role
iam = boto3.client('iam')
ecr = boto3.client('ecr')

sm_role_arn = get_execution_role()
sm_role_name = sm_role_arn.split('/')[-1]
boto_session = boto3.session.Session()
region = boto_session.region_name
account = boto3.client('sts').get_caller_identity()['Account']

In [ ]:
def attach_sm_ecr_policy(sm_role_name):
    iam = boto3.client('iam')
    try:
        policy_response = iam.attach_role_policy(
            RoleName=sm_role_name,
            PolicyArn='arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess'
        )
        return policy_response
    except iam.exceptions.from_code('iam:AttachRolePolicy'):
        print(f'[ERROR] SageMaker is not authorized to perform: iam:AttachRolePolicy on {sm_role_name}. Please add iam policy to this role')    

def attach_private_ecr_policy(repository_name, region, account):
    ecr = boto3.client('ecr')    
    ecr_policy_json = {
      "Version": "2008-10-17",
      "Statement": [
        {
          "Sid": "LambdaECRImageRetrievalPolicy",
          "Effect": "Allow",
          "Principal": {
            "Service": "lambda.amazonaws.com"
          },
          "Action": [
            "ecr:BatchGetImage",
            "ecr:DeleteRepositoryPolicy",
            "ecr:GetDownloadUrlForLayer",
            "ecr:GetRepositoryPolicy",
            "ecr:SetRepositoryPolicy"
          ],
          "Condition": {
            "StringLike": {
              "aws:sourceArn": f"arn:aws:lambda:{region}:{account}:function:*"
            }
          }
        }
      ]
    }
    
    try:
        response = ecr.set_repository_policy(repositoryName=repository_name, policyText=json.dumps(ecr_policy_json))
        return response
    except ecr.exceptions.from_code('AccessDeniedException'):
        print(f'Please add ECR policy on {sm_role_name}')        
    

def create_lambda_role(role_name):
    iam = boto3.client('iam')
    lambda_policy = {
      "Version": "2012-10-17",
      "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "lambda.amazonaws.com"
          },
          "Action": [              
              "sts:AssumeRole"
          ]
        }
      ]
    }    
    
    response = iam.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(lambda_policy)
    )    
    print(response)

    policy_response = iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    return response['Role']['Arn']
    
    
def delete_lambda_role(role_name):
    iam = boto3.client('iam')
    response = iam.detach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    response = iam.delete_role(RoleName=role_name)

### Attach SageMaker policy

In [ ]:
attach_sm_ecr_policy(sm_role_name)

### Create Lambda Role for Serverless Inference

In [ ]:
role_name = 'lambda-role-cv-hol'
repository_name = 'yolov3'
lambda_role_arn = create_lambda_role(role_name)
attach_private_ecr_policy(repository_name, region, account)
time.sleep(10)

<br>

## 2. Deploy & Test
---

도커 이미지가 ECR에 푸시되고 적절한 Lambda Role이 생성되었다면, 단 두 줄의 코드로 `LambdaModel` 및 `LambdaPredictor` 리소스를 순차적으로 생성하여 Serverless Endpoint를 쉽게 생성할 수 있습니다. Serverless Endpoint는 내부적으로 Lambda Container 함수와 동일하므로 Endpoint에 대한 내역을 AWS Console 페이지의 AWS Lambda에서 확인할 수 있으며, 배포 전 Lambda 콘솔 창에서 직접 테스트를 수행할 수도 있습니다. 

### Deploy

In [ ]:
from sagemaker.serverless import LambdaModel
image_uri = f'{account}.dkr.ecr.{region}.amazonaws.com/{repository_name}:latest'
model = LambdaModel(image_uri=image_uri, role=lambda_role_arn)
predictor = model.deploy("my-lambda-function-cv", timeout=50, memory_size=2048)

### Test

Lambda 최초 호출 시 Cold start로 지연 시간이 발생하지만, 최초 호출 이후에는 warm 상태를 유지하기 때문에 빠르게 응답합니다. 물론 수 분 동안 호출이 되지 않거나 요청이 많아지면 cold 상태로 바뀐다는 점을 유의해 주세요.

In [ ]:
def get_cv_prediction(img_path):
    with open(img_path, 'rb') as fp:
        bimage = fp.read()

    input_json = {
        "body": "{\"image\": \"" + base64.b64encode(bimage).decode('utf-8') + "\"}"
    }

    results = predictor.predict(input_json) 
    return json.loads(results['body'])


def load_image_from_base64(img_string, bgr2rgb=True):
    # Decode the base64 string into an image
    base_img = base64.b64decode(img_string)
    npimg = np.frombuffer(base_img, dtype=np.uint8)
    img = cv2.imdecode(npimg, 1)
    if bgr2rgb:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width = img.shape[:2]
    return img, height, width

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img_path = 'sample_images/remote-control.jpeg'
result = get_cv_prediction(img_path)
img, height, width = load_image_from_base64(result['body'])
plt.figure(figsize = (12,12))
plt.imshow(img)

최초 호출 이후에는 빠르게 추론 결과를 얻을 수 있습니다.

In [ ]:
img_path = 'sample_images/remote-control.jpeg'
result = get_cv_prediction(img_path)
img, height, width = load_image_from_base64(result['body'])
plt.figure(figsize = (12,12))
plt.imshow(img)

### Clean-up

테스트를 완료했으면 `delete_model()` 및 `delete_predictor()` 메소드를 사용하여 `LambdaModel` 및 `LambdaPredictor` 리소스를 해제합니다.

In [ ]:
model.delete_model()
predictor.delete_predictor()
delete_lambda_role(role_name)